<a href="https://colab.research.google.com/github/Corosso/ValoAI/blob/main/Project_ValoAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder

# Get the data 
acs = pd.read_csv("teams.csv")
map_win_rate = pd.read_csv("map_win_rate.csv")
partidos_pasados = pd.read_csv("previous_matches.csv")


In [ ]:
# Add a new column to store the team name
acs['team'] = acs['team']

# Split the ACS values into separate columns
acs_s = acs['acs'].str.split(',', expand=True).astype(float)
acs_s.columns = ['acs_1', 'acs_2', 'acs_3', 'acs_4', 'acs_5']

# Concatenate the team name with the ACS values
acs_merged = pd.concat([acs['team'], acs_s], axis=1)



In [ ]:
# Eliminar el número entre paréntesis del nombre del mapa
map_win_rate['map'] = map_win_rate['map'].str.replace(r'\(\d+\)', '')
# Merge ACS and map win rate
data = pd.merge(acs_merged, map_win_rate, on="team")
# Merge provious matches with acs and win rate
data = pd.merge(data, partidos_pasados, left_on="team", right_on="team_A")
# Convert team names to numeric values using label encoder
encoder = LabelEncoder()
data['team'] = encoder.fit_transform(data['team'])
# Convert map names to numeric values using label encoder
encoder = LabelEncoder()
data['map'] = encoder.fit_transform(data['map'])
# Convert team_A and team_B names to numeric values using label encoder
encoder = LabelEncoder()
data['team_A'] = encoder.fit_transform(data['team_A'])
encoder = LabelEncoder()
data['team_B'] = encoder.fit_transform(data['team_B'])
# Reeplace ("-") for NaN
data['win_rate'] = data['win_rate'].replace('-', float('nan'))
# Convert win rate to float
data['win_rate'] = data['win_rate'].str.rstrip('%').astype(float) / 100
# Reeplace NaN for zeros
data = data.fillna(0)

<ipython-input-11-29c3e8d054cc>:2: FutureWarning: The default value of regex will change from True to False in a future version.
  map_win_rate['map'] = map_win_rate['map'].str.replace(r'\(\d+\)', '')


In [ ]:
data

,team,acs_1,acs_2,acs_3,acs_4,acs_5,map,win_rate,team_A,team_B,winner
0,8,186.7,251.5,223.0,220.5,125.0,1,0.68,8,6,1
1,8,186.7,251.5,223.0,220.5,125.0,1,0.68,8,4,0
2,8,186.7,251.5,223.0,220.5,125.0,4,0.81,8,6,1
3,8,186.7,251.5,223.0,220.5,125.0,4,0.81,8,4,0
4,8,186.7,251.5,223.0,220.5,125.0,8,0.75,8,6,1
...,...,...,...,...,...,...,...,...,...,...,...
121,7,250.7,252.7,223.0,186.7,160.0,5,0.64,7,0,1
122,7,250.7,252.7,223.0,186.7,160.0,2,0.71,7,0,1
123,7,250.7,252.7,223.0,186.7,160.0,3,0.57,7,0,1
124,7,250.7,252.7,223.0,186.7,160.0,7,1.00,7,0,1


In [ ]:
# Seleccionar las variables de entrada (ACS y Map Win Rate)
X = data.drop(["team_A", "team_B", "winner"], axis=1)

# Seleccionar la variable de salida (Resultado)
y = data["winner"]

# Dividir los datos en un conjunto de entrenamiento y un conjunto de prueba
X_entrenamiento, X_prueba, y_entrenamiento, y_prueba = train_test_split(X, y, test_size=0.2)

# Crear un modelo de regresión logística
modelo = LogisticRegression(max_iter=10000)

# Entrenar el modelo con los datos de entrenamiento
modelo.fit(X_entrenamiento, y_entrenamiento)

# Evaluar la precisión del modelo con los datos de prueba
precision = modelo.score(X_prueba, y_prueba)
print("Precisión del modelo:", precision)

Precisión del modelo: 0.6923076923076923


In [ ]:
# Make predictions on new data
